# Mumbai Indians: Elite Sports Analytics & Niche Insights

## Overview
This notebook focuses on **Moneyball-style esoteric metrics** and hyper-specific insights that reveal the DNA of Mumbai Indians' success. These are not standard stats; they are advanced performance indicators used by top-tier analysts.

### Niche Insights Covered:
1.  **"The Pollard Effect" (Finisher Efficiency)**: Analyzing Strike Rate acceleration in the last 4 overs.
2.  **Pressure Handling Index (PHI)**: How batters perform when early wickets fall (Powerplay Collapse).
3.  **The "Lucky Charm" Venue**: Win % adjusted for Toss outcome (pure dominance vs luck).
4.  **Bowler Lethality Matrix**: Dot Ball % vs Wicket Taking Ability.
5.  **First Ball Aggression**: Strike Rate on the first 10 balls faced (Starters vs Settlers).
6.  **Milestone Psychology**: Do batters slow down approaching 50s? (Nervous 40s analysis).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Elite Analytics Theme
sns.set_theme(style="darkgrid")
plt.rcParams["figure.figsize"] = (14, 8)
plt.rcParams["font.size"] = 12
MI_BLUE = "#004BA0"
MI_GOLD = "#D1AB3E"

In [ ]:
# --- LOAD DATA ---
# Loading with low_memory=False to ignore warnings
try:
    df_fact = pd.read_csv("MI_Fact_Deliveries.csv", low_memory=False)
    df_matches = pd.read_csv("MI_Dim_Matches.csv", low_memory=False)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("❌ Files not found. Run preparation first.")

## 1. Pressure Handling Index (PHI)
**Insight**: Who rescues MI when the Top Order collapses?
*Methodology*: Filter innings where MI lost >2 wickets in Powerplay (Over 1-6). Identify batters who scored high runs in these specific "Crisis Matches".

In [ ]:
# 1. Identify Matches with Powerplay Collapse
mi_batting = df_fact[df_fact["batting_team"] == "Mumbai Indians"].copy()
pp_wickets = mi_batting[(mi_batting["over"] <= 6) & (mi_batting["is_wicket"] == 1)]

pp_collapse_matches = pp_wickets.groupby("match_id")["is_wicket"].sum()
crisis_matches = pp_collapse_matches[pp_collapse_matches >= 2].index.tolist()

# 2. Analyze Batter Performance in ONLY these matches
crisis_data = mi_batting[mi_batting["match_id"].isin(crisis_matches)]

phi_score = crisis_data.groupby("batter").agg(
    crisis_runs=("runs_batter", "sum"),
    crisis_innings=("match_id", "nunique"),
    crisis_sr=("runs_batter", lambda x: (x.sum() / len(x)) * 100 if len(x) > 0 else 0)
).reset_index()

# Filter for significant sample size
phi_score = phi_score[phi_score["crisis_runs"] > 200].sort_values("crisis_runs", ascending=False).head(10)
phi_score["avg_crisis_score"] = phi_score["crisis_runs"] / phi_score["crisis_innings"]

plt.figure(figsize=(12, 6))
sns.barplot(data=phi_score, x="avg_crisis_score", y="batter", palette="magma")
plt.title("Pressure Handling Index: Avg Runs Scored when MI lost 2+ Wickets in Powerplay", fontsize=16, fontweight='bold')
plt.xlabel("Average Runs in Crisis")
plt.show()

## 2. "The Finisher Effect": Death Over Acceleration
**Insight**: Calculating the **Acceleration Factor**: (Death Over SR) / (Middle Over SR).
High factor = True Finisher. Factor < 1 = Struggles to accelerate.

In [ ]:
# Calculating SR for Middle (7-15) and Death (16-20)
phase_sr = mi_batting.groupby(["batter", "phase"]).agg(
    runs=("runs_batter", "sum"),
    balls=("balls_faced", "sum")
).reset_index()

# Pivot to have phases as columns
phase_pivot = phase_sr.pivot(index="batter", columns="phase", values=["runs", "balls"]).fillna(0)
phase_pivot.columns = ['_'.join(col).strip() for col in phase_pivot.columns.values]

# Filter: Must have faced > 60 balls in BOTH phases
finisher_df = phase_pivot[(
    phase_pivot["balls_Middle (7-15)"] > 60) & 
    (phase_pivot["balls_Death (16-20)"] > 60)
].copy()

# Calculate Strike Rates
finisher_df["SR_Middle"] = (finisher_df["runs_Middle (7-15)"] / finisher_df["balls_Middle (7-15)"]) * 100
finisher_df["SR_Death"] = (finisher_df["runs_Death (16-20)"] / finisher_df["balls_Death (16-20)"]) * 100

# Acceleration Factor
finisher_df["Acceleration"] = finisher_df["SR_Death"] / finisher_df["SR_Middle"]
finisher_df = finisher_df.sort_values("Acceleration", ascending=False).head(10).reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(data=finisher_df, x="Acceleration", y="batter", palette="cool")
plt.axvline(x=1, color='red', linestyle='--', label="No Acceleration")
plt.title("The Finisher Effect: Acceleration Factor (Death SR / Middle SR)", fontsize=16, fontweight='bold')
plt.xlabel("Acceleration Factor (e.g., 1.5 means 50% faster scoring in death)")
plt.show()

## 3. Bowler Lethality Matrix (Dot Ball % vs Wicket %)
**Insight**: Identifying 'Containers' (High Dots, Low Wickets) vs 'Strike Bowlers' (High Wickets, Low Dots) vs 'Elites' (High Both).

In [ ]:
# Load complete dataset to access Bowling Data (since df_fact only has Batting)
try:
    full_df = pd.read_csv("IPL.csv", low_memory=False)
    mi_bowling = full_df[full_df["bowling_team"] == "Mumbai Indians"].copy()

    # Create necessary columns
    if "runs_total" in mi_bowling.columns:
        mi_bowling["is_dot"] = (mi_bowling["runs_total"] == 0).astype(int)
    else:
        mi_bowling["is_dot"] = (mi_bowling["total_runs"] == 0).astype(int)

    bowler_lethality = mi_bowling.groupby("bowler").agg(
        balls=("valid_ball", "sum"),
        dots=("is_dot", "sum"),
        wickets=("bowler_wicket", "sum")
    ).reset_index()

    bowler_lethality = bowler_lethality[bowler_lethality["balls"] > 300].copy()

    bowler_lethality["dot_pct"] = (bowler_lethality["dots"] / bowler_lethality["balls"]) * 100
    bowler_lethality["wicket_pct"] = (bowler_lethality["wickets"] / bowler_lethality["balls"]) * 100

    plt.figure(figsize=(12, 8))
    sns.scatterplot(
        data=bowler_lethality, 
        x="dot_pct", 
        y="wicket_pct", 
        size="wickets",
        sizes=(100, 800),
        color="#2ca02c"
    )

    for line in range(0, bowler_lethality.shape[0]):
        plt.text(
            bowler_lethality.dot_pct.iloc[line]+0.2, 
            bowler_lethality.wicket_pct.iloc[line], 
            bowler_lethality.bowler.iloc[line], 
            size='small'
        )

    plt.title("Bowler Lethality Matrix: Dot Ball % vs Wicket Taking %", fontsize=16, fontweight='bold')
    plt.xlabel("Dot Ball Percentage (Control)")
    plt.ylabel("Wicket Percentage (Lethality)")
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.show()

except FileNotFoundError:
    print("IPL.csv not found. Cannot perform Bowler Analysis.")
except KeyError as e:
    print(f"Column missing: {e}. Check IPL.csv structure.")


## 4. "Nervous Nineties?" (The 40s Analysis)
**Insight**: Do MI Batters slow down when approaching a 50?
Compare SR when score is 30-39 vs 40-49.

In [ ]:
# Calculate cumulative run count for each batter in each match
mi_batting["curr_runs"] = mi_batting.groupby(["match_id", "batter"])["runs_batter"].cumsum()

# Define score bucket
def score_bucket(r):
    if 30 <= r <= 39: return "30s"
    elif 40 <= r <= 49: return "40s"
    else: return "Other"

mi_batting["score_range"] = mi_batting["curr_runs"].apply(score_bucket)

# Filter for runs in these buckets
nervous_data = mi_batting[mi_batting["score_range"].isin(["30s", "40s"])]

nervous_stats = nervous_data.groupby(["batter", "score_range"]).agg(
    runs=("runs_batter", "sum"),
    balls=("balls_faced", "sum")
).reset_index()

nervous_stats["SR"] = (nervous_stats["runs"] / nervous_stats["balls"]) * 100

# Filter for top players
top_players = ["RG Sharma", "KA Pollard", "SK Yadav", "AT Rayudu", "Ishan Kishan"]
nervous_stats = nervous_stats[nervous_stats["batter"].isin(top_players)]

plt.figure(figsize=(12, 6))
sns.barplot(data=nervous_stats, x="batter", y="SR", hue="score_range", palette="Set2")
plt.title("The Nervous 40s: Strike Rate Comparison (30s vs 40s)", fontsize=16, fontweight='bold')
plt.ylabel("Strike Rate")
plt.legend(title="Score Range")
plt.show()

## 5. First 10 Balls Aggression (Starters vs Settlers)
**Insight**: Who attacks from Ball 1? Who takes time to settle?
*Methodology*: Calculate SR for the first 10 balls faced in every innings.

In [ ]:
# We need ball number for each batter within an innings
# Create a counter grouping by match and batter
mi_batting["batter_ball_no"] = mi_batting.groupby(["match_id", "batter"]).cumcount() + 1

# Filter for first 10 balls
first_10 = mi_batting[mi_batting["batter_ball_no"] <= 10]

start_stats = first_10.groupby("batter").agg(
    runs=("runs_batter", "sum"),
    balls=("balls_faced", "sum")
).reset_index()

# Filter significant innings
start_stats = start_stats[start_stats["balls"] > 100].copy()
start_stats["Start_SR"] = (start_stats["runs"] / start_stats["balls"]) * 100
start_stats = start_stats.sort_values("Start_SR", ascending=False).head(15)

plt.figure(figsize=(12, 6))
sns.barplot(data=start_stats, x="Start_SR", y="batter", palette="autumn")
plt.title("Aggressive Starters: Strike Rate in First 10 Balls Faced", fontsize=16, fontweight='bold')
plt.xlabel("Strike Rate (First 10 Balls)")
plt.show()

## 6. "The Lucky Charm" Venue (Toss vs Win Analysis)
**Insight**: Identifying venues where Mumbai Indians' win percentage is heavily dependent on winning the toss (High Luck Factor) vs venues where they dominate regardless (Fortresses).

In [ ]:
# 1. Prepare Match Data
df_matches["mi_toss_win"] = (df_matches["toss_winner"] == "Mumbai Indians").astype(int)
df_matches["mi_win"] = (df_matches["match_won_by"] == "Mumbai Indians").astype(int)

# 2. Calculate Stats by Venue
venue_luck = df_matches.groupby("venue").agg(
    matches=("match_id", "count"),
    toss_wins=("mi_toss_win", "sum"),
    wins=("mi_win", "sum"),
    wins_toss_win=("mi_win", lambda x: x[df_matches.loc[x.index, "mi_toss_win"] == 1].sum()),
    wins_toss_loss=("mi_win", lambda x: x[df_matches.loc[x.index, "mi_toss_win"] == 0].sum())
).reset_index()

# Filter for significant venues (> 5 matches)
venue_luck = venue_luck[venue_luck["matches"] >= 5].copy()

# 3. Calculate Percentages
venue_luck["win_pct_toss_win"] = (venue_luck["wins_toss_win"] / venue_luck["toss_wins"]) * 100
venue_luck["toss_losses"] = venue_luck["matches"] - venue_luck["toss_wins"]
venue_luck["win_pct_toss_loss"] = (venue_luck["wins_toss_loss"] / venue_luck["toss_losses"]) * 100

# Handle edge cases (divide by zero) with fillna
venue_luck = venue_luck.fillna(0)

# 4. Define Luck Factor (Difference)
venue_luck["Toss_Dependence"] = venue_luck["win_pct_toss_win"] - venue_luck["win_pct_toss_loss"]
venue_luck = venue_luck.sort_values("Toss_Dependence", ascending=False).head(10)

plt.figure(figsize=(12, 6))
sns.barplot(data=venue_luck, x="Toss_Dependence", y="venue", palette="viridis")
plt.title("The Lucky Charm: Toss Dependence (Win % Diff: Toss Won vs Lost)", fontsize=16, fontweight='bold')
plt.xlabel("Percent Difference (Higher = Needs Toss to Win)")
plt.axvline(x=0, color='black', linestyle='--')
plt.show()

## 7. Champions DNA: Winning vs Non-Winning Seasons
**Insight**: What statistical anomalies separate MI's title-winning seasons (2013, 2015, 2017, 2019, 2020) from the rest?
- **Win %**: How dominant were they?
- **Batting Stability**: Average runs per wicket.
- **Bowling Discipline**: Bowling Average comparison.

In [ ]:
# 1. Define Winning Years and Clean Season Data
WINNING_YEARS = [2013, 2015, 2017, 2019, 2020]

def clean_season(s):
    s = str(s)
    if "/" in s:
        return int(s.split("/")[0])
    return int(s)

# Apply to Batting Data
if "season" in mi_batting.columns:
    mi_batting["season_year"] = mi_batting["season"].apply(clean_season)
    mi_batting["is_champion"] = mi_batting["season_year"].isin(WINNING_YEARS)

# Apply to Bowling Data
if "season" in mi_bowling.columns:
    mi_bowling["season_year"] = mi_bowling["season"].apply(clean_season)
    mi_bowling["is_champion"] = mi_bowling["season_year"].isin(WINNING_YEARS)

# 2. Calculate Metrics
# Batting Stats
bat_dna = mi_batting.groupby("is_champion").agg(
    runs=("runs_batter", "sum"),
    wickets=("is_wicket", "sum"),
    balls=("valid_ball", "sum")
).reset_index()
bat_dna["Batting Avg"] = bat_dna["runs"] / bat_dna["wickets"]
bat_dna["Batting SR"] = (bat_dna["runs"] / bat_dna["balls"]) * 100

# Bowling Stats
bowl_dna = mi_bowling.groupby("is_champion").agg(
    runs_conceded=("runs_total", "sum"),
    wickets=("bowler_wicket", "sum"),
    balls=("valid_ball", "sum")
).reset_index()
bowl_dna["Bowling Avg"] = bowl_dna["runs_conceded"] / bowl_dna["wickets"]
bowl_dna["Economy"] = (bowl_dna["runs_conceded"] / bowl_dna["balls"]) * 6

# Merge for Visualization
dna_comparison = pd.merge(bat_dna, bowl_dna, on="is_champion")
dna_comparison["Status"] = dna_comparison["is_champion"].map({True: "Champion Years", False: "Other Years"})

# 3. Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

sns.barplot(data=dna_comparison, x="Status", y="Batting Avg", ax=axes[0], palette="Blues")
axes[0].set_title("Batting Stability (Average)", fontweight="bold")

sns.barplot(data=dna_comparison, x="Status", y="Bowling Avg", ax=axes[1], palette="Reds")
axes[1].set_title("Bowling Lethality (Runs per Wicket)", fontweight="bold")
# Note: Lower Bowling Avg is better

sns.barplot(data=dna_comparison, x="Status", y="Economy", ax=axes[2], palette="Greens")
axes[2].set_title("Bowling Control (Economy Rate)", fontweight="bold")

plt.suptitle("Champions DNA: Statistical Comparison", fontsize=20, fontweight="bold")
plt.tight_layout()
plt.show()